# SINDy Exercises

In this notebook we will use the library *PySINDy* to construct our SINDy models. You can install PySINDY with *pip* in your anaconda environment. To do this, you should install *pip* in your conda environment. After doing this, you can install PySINDy with the following command:

*/anaconda/envs/\<your_env_name\>/bin/pip install pysindy*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pysindy as ps

# Exercise 1

## Linear 2D ODE

**(This exercise comes from the examples provided by the PySINDy documentation).** 
We consider the following linear system of differential equations:

\begin{align}
\frac{d}{dt} \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} -0.1 & 2 \\ -2 & -0.1 \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix}
\end{align}

which describes a linear two-dimensional damped harmonic oscillator.

We generate training data by integrating the linear system of differential equations with initial condtion $(2, 0)$. To apply SINDy, we need to gather measurements of the state $(x, y)$ and the first-order derivatives $(\dot{x}, \dot{y})$.

In [ ]:
from scipy.integrate import solve_ivp
from pysindy.utils import linear_damped_SHO

# Integrator keywords for solve_ivp
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

# Generate training data (state measurements)

dt = 0.01
t_train = np.arange(0, 25, dt)
t_train_span = (t_train[0], t_train[-1])

x0_train = [2, 0] # Initial condition
x_train = solve_ivp(linear_damped_SHO, t_train_span,
                    x0_train, t_eval=t_train, **integrator_keywords).y.T

# Print the first 10 sample of the dataset
print(x_train[:10])

In [ ]:
# EXERCISE
# Generate training data (first-derivative measurements)

x_train_dot = []
for state in x_train:
    # You can obtain the corresponding first-derivative measurements by applying the linear system above
    # TO DO

x_train_dot = np.array(x_train_dot)

We will compose our library with polynomial functions up to order 5. To construct the model with PySINDy, we can specify the features names (in this case, we call them $x$ and $y$), we have define the optimizer to solve our problem (we will be using the *Sequentially Thresholded Least Squares algorithm*) and we can construct the polynomial library with the function *PolynomialLibrary(degree=...)* from PySINDy.

In [ ]:
poly_order = 5
threshold = 0.05

# We can use PolynomialLibrary function to generate a library with polynomial functions
model = ps.SINDy(
    feature_names=["x", "y"],
    optimizer=ps.STLSQ(threshold=threshold), # Sequentially Thresholded Least Squares algorithm
    feature_library=ps.PolynomialLibrary(degree=poly_order),
)
model.fit(x_train, 
          t=dt, 
          x_dot=x_train_dot)
model.print()

And we see that the model was able to return the true governing equations. The learned model can be used to evolve initial conditions forward in time. To do this, you can use the function *model.simulate(x0_train, t_train)* by giving the initial state (*x0_train*) and the timesteps we wish to propagate the system (*t_train*).

In [ ]:
# Simulate and plot the results

x_sim = model.simulate(x0_train, t_train)
plot_kws = dict(linewidth=2)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].plot(t_train, x_train[:, 0], "r", label="$x_0$", **plot_kws)
axs[0].plot(t_train, x_train[:, 1], "b", label="$x_1$", alpha=0.4, **plot_kws)
axs[0].plot(t_train, x_sim[:, 0], "k--", label="model", **plot_kws)
axs[0].plot(t_train, x_sim[:, 1], "k--")
axs[0].legend()
axs[0].set(xlabel="t", ylabel="$x_k$")

axs[1].plot(x_train[:, 0], x_train[:, 1], "r", label="$x_k$", **plot_kws)
axs[1].plot(x_sim[:, 0], x_sim[:, 1], "k--", label="model", **plot_kws)
axs[1].legend()
axs[1].set(xlabel="$x_1$", ylabel="$x_2$")

### Automatic Differentiation

If we do not have access to measures of the first-order derivative of the state, PySINDy can extract them through automatic differentiation. In this case, we just have to fit the model without specifying the *x_dot* field.

In [ ]:
# Fit the model

poly_order = 5
threshold = 0.05

# We can use PolynomialLibrary function to generate a library with polynomial functions
model = ps.SINDy(
    feature_names=["x", "y"],
    optimizer=ps.STLSQ(threshold=threshold),
    feature_library=ps.PolynomialLibrary(degree=poly_order),
)
# Here, we do not specify the x_dot field
model.fit(x_train, t=dt)
model.print()

In [ ]:
# Simulate and plot the results

x_sim = model.simulate(x0_train, t_train)
plot_kws = dict(linewidth=2)

fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].plot(t_train, x_train[:, 0], "r", label="$x$", **plot_kws)
axs[0].plot(t_train, x_train[:, 1], "b", label="$y$", alpha=0.4, **plot_kws)
axs[0].plot(t_train, x_sim[:, 0], "k--", label="model", **plot_kws)
axs[0].plot(t_train, x_sim[:, 1], "k--")
axs[0].legend()
axs[0].set(xlabel="t", ylabel="$x_k$")

axs[1].plot(x_train[:, 0], x_train[:, 1], "r", label="true system", **plot_kws)
axs[1].plot(x_sim[:, 0], x_sim[:, 1], "k--", label="model", **plot_kws)
axs[1].legend()
axs[1].set(xlabel="$x$", ylabel="$y$")

# Exercise 2

## Lorenz System (Chaotic Behaviour)

The Lorenz system serves as an example of a nonlinear ODE whose solutions exhibit chaotic dynamics evolving on a strange attractor. The Lorenz system is given by 

\begin{align}
\dot{x} &= \sigma (y-x) \\
\dot{y} &= x (\rho - z) - y \\
\dot{z} &= x y - \beta z
\end{align}

with $\sigma = 10$, $\rho = 28$ and $\beta = \frac{8}{3}$ for this example. We generate our training data starting from the initial condition $(-8, 8, 27)$.

In [ ]:
from scipy.integrate import solve_ivp
from pysindy.utils import lorenz

# Integrator keywords for solve_ivp
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

dt = 0.001
t_train = np.arange(0, 100, dt)
t_train_span = (t_train[0], t_train[-1])
x0_train = [-8, 8, 27] # Initial condition

# Generate training data (state measurements)
x_train = solve_ivp(lorenz, t_train_span,
                    x0_train, t_eval=t_train, **integrator_keywords).y.T

# Generate training data (first derivative measurements)
x_train_dot = np.array(
    [lorenz(0, x_train[i]) for i in range(t_train.size)]
)

Construct a SINDy model with a polynomial library of degree 5. You should use the same optimizer as before, *STLSQ*, with a threshold of $0.05$.

In [ ]:
# EXERCISE


In [ ]:
# Simulate and plot the results
t_sim = np.arange(0, 20, dt)
x_sim = model.simulate(x_train[0], t_sim)


fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot(131, projection="3d")
ax.plot(
    x_train[: t_sim.size, 0],
    x_train[: t_sim.size, 1],
    x_train[: t_sim.size, 2],
)
plt.title("full simulation")
ax.set(xlabel="$x$", ylabel="$y$", zlabel="$z$")

ax = fig.add_subplot(132, projection="3d")
ax.plot(x_sim[:, 0], x_sim[:, 1], x_sim[:, 2])
plt.title(f"identified system")
ax.set(xlabel="$x$", ylabel="$y$", zlabel="$z$")

In [ ]:
# Plot each axis independently

fig = plt.figure(figsize=(12, 5))

ax = fig.add_subplot(311)
ax.plot(t_sim, x_train[: t_sim.size, 0], "r", label="true system")
ax.plot(t_sim, x_sim[:, 0], "k--", label="model")
plt.ylabel("x")

ax.legend()

ax = fig.add_subplot(312)
ax.plot(t_sim, x_train[: t_sim.size, 1], "r", label="true system")
ax.plot(t_sim, x_sim[:, 1], "k--", label="model")
plt.ylabel("y")

ax.legend()

ax = fig.add_subplot(313)
ax.plot(t_sim, x_train[: t_sim.size, 2], "r", label="true system")
ax.plot(t_sim, x_sim[:, 2], "k--", label="model")
plt.ylabel("z")
plt.xlabel("time")

ax.legend()

### Impact of noise with automatic differentiation

In this example, only the states, $(x,y,z)$, are measured and noise at different levels is added to the states. Derivatives are computed from the noisy state measurements using a centered difference scheme.

In [ ]:
noise_levels = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]

t_sim = np.arange(0, 20, dt)
models_noD = []
x_sim_noD = []

for noise in noise_levels:
    # Generate noisy state measurements
    x_train_noise = x_train + np.random.normal(scale=noise, size=x_train.shape)
    
    # EXERCISE
    # Construct a SINDy model with a polynomial library of degree 5
    # TO DO
    
    # Fit the noisy training data
    # TO DO
    
    models_noD.append(model)
    x_sim_noD.append(model.simulate(x_train[0], t_sim))

In [ ]:
fig = plt.figure(figsize=(12, 5))

model_idx = 0
ax = fig.add_subplot(321)
ax.plot(t_sim, x_train[: t_sim.size, 0], "r", label="true system")
ax.plot(t_sim, x_sim_noD[model_idx][:, 0], "k--", label="model")
plt.title(f"$\eta$={noise_levels[model_idx]:.1e}")
plt.ylabel("x")

ax.legend()

ax = fig.add_subplot(323)
ax.plot(t_sim, x_train[: t_sim.size, 1], "r", label="true system")
ax.plot(t_sim, x_sim_noD[model_idx][:, 1], "k--", label="model")
plt.ylabel("y")

ax.legend()

ax = fig.add_subplot(325)
ax.plot(t_sim, x_train[: t_sim.size, 2], "r", label="true system")
ax.plot(t_sim, x_sim_noD[model_idx][:, 2], "k--", label="model")
plt.ylabel("z")

ax.legend()

plt.xlabel("time")

model_idx = -1
ax = fig.add_subplot(322)
ax.plot(t_sim, x_train[: t_sim.size, 0], "r", label="true system")
ax.plot(t_sim, x_sim_noD[model_idx][:, 0], "k--", label="model")
plt.title(f"$\eta$={noise_levels[model_idx]:.1e}")
plt.ylabel("x")

ax.legend()

ax = fig.add_subplot(324)
ax.plot(t_sim, x_train[: t_sim.size, 1], "r", label="true system")
ax.plot(t_sim, x_sim_noD[model_idx][:, 1], "k--", label="model")
plt.ylabel("y")

ax.legend()

ax = fig.add_subplot(326)
ax.plot(t_sim, x_train[: t_sim.size, 2], "r", label="true system")
ax.plot(t_sim, x_sim_noD[model_idx][:, 2], "k--", label="model")
plt.ylabel("z")

ax.legend()

plt.xlabel("time")

# Exercise 3

## Orbital Motion

Consider a system containing one planet at the origin and a satellite orbiting around. The planetary motion is described by the following differential equation:

\begin{align}
\ddot{\mathbf{r}} &= - \frac{\mu \mathbf{r}}{\|\mathbf{r}\|^3}
\end{align}

such that $\mathbf{r}$ denotes the position of the satellite and $\mu$ denotes the standard gravitational parameter. We generate our training data by integrating the differential equation

\begin{align}
\begin{bmatrix} \dot{\mathbf{r}} \\ \ddot{\mathbf{r}} \end{bmatrix} = \begin{bmatrix} \mathbf{v} \\ - \frac{\mu \mathbf{r}}{\|\mathbf{r}\|^3} \end{bmatrix}
\end{align}


and considering the inital position, $\mathbf{r} = (1, 0)$, and the inicial velocity, $\mathbf{v} = (0, 0.1)$, which is the first-order derivative of the position. For this exercise, we assume we have access to measurements of the position and velocity of the satellite throughout its trajectory.

Start by complete the function *orbital_differential_equation* which defines the differential equation that describes the dynamics. We assume that the *state* parameter contains the position and velocity of the object, *state = (x, y, v_x, v_y)*.

In [ ]:
def orbital_differential_equation(t, state, mu):
    # Exercise: Define the function to integrate in order to 
    # obtain the satellite trajectory given an initial state
    # with both position and velocity
    
    return ...

In [ ]:
# Integrator keywords for solve_ivp
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

# Generate training data (state measurements)

def generate_data(r0, v0, mu=1.0, integrator_keywords=integrator_keywords):
    dt = 0.01
    t_train = np.arange(0, 2500, dt)
    t_train_span = (t_train[0], t_train[-1])
    
    x0_train = np.concatenate((r0, v0)) # Initial state
    x_train = solve_ivp(orbital_differential_equation, t_train_span,
                    x0_train, t_eval=t_train, args=(mu,), **integrator_keywords).y.T
    
    X = []
    timestamps = []
    for i in range(x_train.shape[0]):
        if i % 100 == 0:
            X.append(x_train[i])
            timestamps.append(t_train[i])
    X = np.array(X)
    timestamps = np.array(timestamps)
    
    return np.concatenate((timestamps.reshape(-1,1), X), axis=1)


# Generate training data (first-derivative measurements)

def generate_first_derivative_data(x_train, mu=1.0):

    x_train_dot = []
    for state in x_train:
        pos = state[1:3]
        vel = state[3:]

        r_ddot = -mu * pos / np.linalg.norm(pos)**3

        first_der = np.array([vel[0], vel[1], r_ddot[0], r_ddot[1]])

        x_train_dot.append(first_der.flatten())

    return np.array(x_train_dot)

In [ ]:
r0=[2.0, 0] # Initial position
v0=[0, 0.6] # Initial velocity

x0_train = np.concatenate((r0, v0)) # Initial state

mu = 1.0
x_train = generate_data(r0, v0, mu)
x_train_dot = generate_first_derivative_data(x_train, mu)

In [ ]:
# Visualize the trajectory of the object
timesteps = x_train[:, 0]
positions = x_train[:, 1:3]
velocities = x_train[:, 3:]

plt.plot(positions[:, 0], positions[:, 1], 'o', markersize=0.4, label='Trajectory')
plt.plot(0, 0, 'o', label='Planet')
plt.legend(loc='upper right')
plt.xlabel('$x$')
plt.ylabel('$y$')

Construct a SINDy model with a polynomial library of degree 5. You should use the same optimizer as before, *STLSQ*, with a threshold of $0.05$.

In [ ]:
# EXERCISE


In [ ]:
# Simulate and plot the results

x_sim = model.simulate(x0_train, timesteps)

plt.plot(positions[:,0], positions[:,1], "ro", markersize=0.6, label="true system")
plt.plot(x_sim[:,0], x_sim[:,1], "ko", markersize=0.6, label="model")
plt.legend(loc='upper right')
plt.xlabel('$x$')
plt.ylabel('$y$')

In [ ]:
plt.plot(timesteps[:100], positions[:100,0], label='$x$')
plt.plot(timesteps[:100], x_sim[:100,0], 'k--', label='model')
plt.plot(timesteps[:100], positions[:100,1], label='$y$')
plt.plot(timesteps[:100], x_sim[:100,1], 'k--')
plt.legend()

### Custom library

In this exercise, we define a custom library of function with the correct expression of the differential equation

\begin{align}
\begin{bmatrix} \dot{\mathbf{r}} \\ \ddot{\mathbf{r}} \end{bmatrix} = \begin{bmatrix} \mathbf{v} \\ - \frac{\mathbf{r}}{\|\mathbf{r}\|^3} \end{bmatrix}
\end{align}

and see if SYNDy can identify the parameter $\mu$. Change the parameter $\mu$ to different values and see how the model is able to obtain the correct governing equations

In [ ]:
# Integrator keywords for solve_ivp
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

# Generate training data (state measurements)

mu = 5.0 # You can change this parameter and see if the model is able to capture its value
r0 = [2.0, 0]
v0 = [0, 0.6]

x0_train = np.concatenate((r0, v0)) # Initial state
x_train = generate_data(r0, v0, mu)

# Generate training data (first-derivative measurements)

x_train_dot = generate_first_derivative_data(x_train, mu)

In [ ]:
positions = x_train[:, 1:3]
velocities = x_train[:, 3:]

plt.plot(positions[:, 0], positions[:, 1], 'o', markersize=0.4, label='Trajectories')
plt.plot(0, 0, 'o', label='Planet')
plt.legend(loc='upper right')

In [ ]:
# Fit the model

poly_order = 5
threshold = 0.01

# Custom library of functions
functions = [
    lambda x, y, v_x, v_y : v_x,
    lambda x, y, v_x, v_y : v_y,
    lambda x, y, v_x, v_y : (-x / (x**2 + y**2)**(3/2)), 
    lambda x, y, v_x, v_y : (-y / (x**2 + y**2)**(3/2))
]

library_function_names = [
    lambda x, y, v_x, v_y: "v_x",
    lambda x, y, v_x, v_y: "v_y",
    lambda x, y, v_x, v_y: "(- x / ||r||^3)",
    lambda x, y, v_x, v_y: "(- y / ||r||^3)"
]


model = ps.SINDy(
    feature_names=["x", "y", "v_x", "v_y"],
    optimizer=ps.STLSQ(threshold=threshold),
    # We now use our custom library instead of the Polynomial library
    feature_library=ps.CustomLibrary(
        library_functions=functions,
        function_names=library_function_names
    )
)
model.fit(x_train[:,1:], t=dt, x_dot=x_train_dot)
model.print()

We see that the model is able to obtain the corret equations. The first-order derivative of the position is the velocity. The first-oder derivative of the velocity is a unit vector with the opposite direction of the position vector with the gravitational parameter $mu$ correctly quantified.

In [ ]:
# Simulate and plot the results

x_sim = model.simulate(x0_train, timesteps)

plt.plot(positions[:,0], positions[:,1], "ro", markersize=0.6, label="true system")
plt.plot(x_sim[:,0], x_sim[:,1], "ko", markersize=0.6, label="model")
plt.legend(loc='upper right')
plt.xlabel('$x$')
plt.ylabel('$y$')

In [ ]:
plt.plot(timesteps[:100], positions[:100,0], label='$x$')
plt.plot(timesteps[:100], x_sim[:100,0], 'k--', label='model')
plt.plot(timesteps[:100], positions[:100,1], label='$y$')
plt.plot(timesteps[:100], x_sim[:100,1], 'k--')
plt.legend()